# SQL Injection

### The purpose of this exercise is to demonstrate how unsanitized database queries can lead to leaks of personal information. 

### In this exercise we will:

<ul>
<li>Connect to a database.</li>
<li>Submit a legitimate query.</li>
<li>Submit a variety of illicit queries.</li>
<li>Demonstrate how to sanitize database inputs to reduce the occurance of SQL injection attacks.</li>
</ul>

A sample SQLite database has been created that stores information about a set of employees, including their names, social security numbers, home address, and other information.  The follow code snippet demonstrates how to connect to the database and query that infomration using the SQL _SELECT_ statement.

In [ ]:
from background import *

connection = create_connection("test.db")

select_users = "SELECT * from EMPLOYEE"
users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

There are situations where one may not wish a user to be able to see information about arbitrary entires in the database.  For instance, the next code snippet requires a user to enter their SSN in order to see their current information.  The assumption here is that one only knows their own SSN, and not the SSN of their co-workers.  Try the example for the employee whose SSN is 123456789.

In [ ]:
ssn="123456789"

In [ ]:
select_users = "SELECT * from employee where ssn=" + ssn

users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

Very good.  Now a user can only see information for SSN numbers they know.  There is a first, obvious, vulerability in that a user could guess random SSNs and, perhaps, leak information about their co-workers, but the search space is large.  A second, more insidious vulnerable is known as _SQL Injection_ where additional SQL code can be fed into the application to change the results.  Append "OR TRUE" to the SSN and run the sample again to see what happens.

As you can see, because of the mechanism the code uses to build their query, it is vulnerable to manipulation of the where clause.  Let's run a new version that _sanitizes_ the user input to ensure it only contains a valid SSN.

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png">

In [ ]:
if(isSSN(ssn)):
    select_users = "SELECT * from employee where ssn=" + ssn

    users = execute_read_query(connection, select_users)

    for user in users:
        print(user) 

Try running it with an without the "OR TRUE" and you'll see only the valid SSN works.

Let's try another example.  Let's say you have the following script to add new employees to the database.  Can you spot the vulnerability?

In [ ]:
first_name = "Abraham"
last_name = "Lincoln"
ssn = "222222222"
department_number = "1";

add_user = """
INSERT INTO EMPLOYEE (Fname,Lname,Ssn,Dno)
VALUES
(\"""" + first_name + "\",\"" + last_name + "\"," + ssn + "," + department_number + ");"

execute_queries(connection,add_user)


At first, it might seem like there is no good way to good way to alter the query to expose extra information, because it is constrainted by the values in the insert clause.  However, if you notice, the writer of the script allows multiple queries to be run at once.

This allows us to use another class of exploits that involves appending additional queries and running them all at the same time.  Look at the following code sample and see what the fully resolved query looks like when printed.

In [ ]:
first_name = "George"
last_name = "Washington"
ssn = "555555555"
department_number = """1);

UPDATE EMPLOYEE
SET Salary=100000
WHERE Lname="English";

DELETE FROM EMPLOYEE
WHERE (Lname="Washington\""""

add_user = """
INSERT INTO EMPLOYEE (Fname,Lname,Ssn,Dno)
VALUES
(\"""" + first_name + "\",\"" + last_name + "\"," + ssn + "," + department_number + ");"

print(add_user)

You can see that with some clever formating we have spliced two additional queries into a single string, adding a dummy employee, adjusting an employee's salary, and removing the dummy employee.

Run the following code to use the exploit query and print the entire table.  Compare the table to our original printout and see what changed.

In [ ]:
execute_queries(connection,add_user)

select_users = "SELECT * from employee"
users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

If you compare this to the original table, you will see that Joyce has received a nice raise.  While it is convient to allow execution of batches to queries to, for instance, initialize your database, you should restrict execution to one query at a time in public facing scripts.

In summary, if you construct a dynamic SQL query based on user input make sure to sanitize your inputs!